In [1]:
import pandas as pd
import requests
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import os
import ast

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

max_seq_length = 2048
lora_rank       = 8

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name            = 'grpo_saved_lora_3B',
    max_seq_length        = max_seq_length,
    load_in_4bit          = True,    # 4‑bit base + LoRA
    fast_inference        = True,
    max_lora_rank         = lora_rank,
    gpu_memory_utilization= 0.9,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-21 20:19:44.779046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745291984.797516  749656 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745291984.803370  749656 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745291984.818072  749656 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745291984.818086  749656 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745291984.818088  749656 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-21 20:19:51 __init__.py:190] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.2.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Your GPU does not support prefix caching - will disable!
Unsloth: vLLM loading unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 89.03%
Unsloth: Your GPU has CUDA compute capability 7.0 with VRAM = 31.73 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 25.8

[W421 20:20:12.018379907 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-21 20:20:13 weight_utils.py:252] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-21 20:20:15 model_runner.py:1115] Loading model weights took 2.2405 GB
INFO 04-21 20:20:15 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-21 20:20:17 worker.py:267] Memory profiling takes 2.13 seconds
INFO 04-21 20:20:17 worker.py:267] the current vLLM instance can use total_gpu_memory (31.73GiB) x gpu_memory_utilization (0.89) = 28.25GiB
INFO 04-21 20:20:17 worker.py:267] model weights take 2.24GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 1.49GiB; the rest of the memory reserved for KV Cache is 24.45GiB.
INFO 04-21 20:20:18 executor_base.py:110] # CUDA blocks: 14309, # CPU blocks: 3510
INFO 04-21 20:20:18 executor_base.py:115] Maximum concurrency for 2048 tokens per request: 111.79x
INFO 04-21 20:20:21 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error

Capturing CUDA graph shapes: 100%|██████████| 43/43 [00:43<00:00,  1.00s/it]

INFO 04-21 20:21:04 model_runner.py:1562] Graph capturing finished in 43 secs, took 0.76 GiB
INFO 04-21 20:21:04 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 49.58 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
# PROMPTING FUNCTION
def llm_output(words):
  prompt = '''You are playing the NY Times Connections game. Your task is to categorize 16 given words into exactly 4 groups of 4 words each, based on shared common themes.

Solve the puzzle using these clear steps:

1. THINK STEP-BY-STEP: Begin by carefully analyzing the words within <thinking> tags. Identify their meanings, relationships, and possible groupings logically.
Example:
<thinking>To solve this, I will first look for obvious groupings like NBA teams, palindrome words, weather-related words, or computer keys. Then, I'll group these words accordingly and ensure each word belongs to exactly one group.</thinking>

2. PROVIDE FINAL ANSWER: After clearly grouping and justifying all four sets, provide ONLY your final solution within <answer> tags. Format your solution exactly as shown below.

Example:
<answer>
[['HAIL', 'RAIN', 'SLEET', 'SNOW'],
 ['BUCKS', 'HEAT', 'JAZZ', 'NETS'],
 ['OPTION', 'RETURN', 'SHIFT', 'TAB'],
 ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]
</answer>

Important Notes:
- Categories should be specific
- Words cannot appear in more than one group.
- Categories can include compound words, shared prefixes/suffixes, pop culture references, or common phrases.
- DO NOT ADD NEW WORDS THAT ARE NOT MENTIONED IN THE QUESTION. USE ONLY WORDS MENTIONED AND GROUP THEM

Here is an example:

USER: [BUCKS, HAIL, JAZZ, SHIFT, LEVEL, MOM, SNOW, RACECAR, SLEET, TAB, KAYAK, RETURN, OPTION, NETS, RAIN, HEAT]

<answer>
[['HAIL', 'RAIN', 'SLEET', 'SNOW'],
 ['BUCKS', 'HEAT', 'JAZZ', 'NETS'],
 ['OPTION', 'RETURN', 'SHIFT', 'TAB'],
 ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]
</answer>

Explanation:
- WEATHER TERMS: 'HAIL', 'RAIN', 'SLEET', 'SNOW'
- NBA TEAMS: 'BUCKS', 'HEAT', 'JAZZ', 'NETS'
- KEYBOARD KEYS: 'OPTION', 'RETURN', 'SHIFT', 'TAB'
- PALINDROMES: 'KAYAK', 'LEVEL', 'MOM', 'RACECAR'
Give the answer for the following set of words -''' + str(words)

  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

  output_ids = model.generate(**inputs, max_length=1024, temperature=0.7, do_sample=True)

  response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # print("\nResponse:", response)
  return response

In [16]:
# EVALUATION FUNCTION
def evaluation_scores(actual_clusters, predicted_clusters):
  if len(actual_clusters) == 0 or len(predicted_clusters) == 0:
    return 0
  score_matrix = np.zeros((len(actual_clusters), len(predicted_clusters)))
  jacc_similarity_score = 0

  # Evauating jaccard matching between each pair of sets
  for actual_idx in range(4):
    for predicted_idx in range(4):
      actual_group = actual_clusters[actual_idx]
      predicted_group = predicted_clusters[predicted_idx]
      score_matrix[actual_idx, predicted_idx] = len(set(actual_group) & set(predicted_group)) / len(set(actual_group) | set(predicted_group))

  # Success Rate
  succ_rate = np.sum(np.array(score_matrix) == 1.0)

  total_words = sum(len(cluster) for cluster in predicted_clusters)  # Should be 16 in this case
  max_overlaps = 0
  for pred_cluster in predicted_clusters:
      overlap_counts = Counter()
      for true_cluster in actual_clusters:
          overlap_counts[tuple(true_cluster)] = len(set(pred_cluster) & set(true_cluster))

      max_overlaps += max(overlap_counts.values())  # Take the largest overlap for each predicted cluster
  # print("SUCCESS RATE: ", succ_rate)
  return succ_rate , max_overlaps / total_words

def evaluation(actual_clusters_set, predicted_clusters_set):
  succ_rate = 0
  purity = 0
  num_samples = 0
  for actual_clusters, predicted_clusters in zip(actual_clusters_set, predicted_clusters_set):
    num_samples += 1
    # print("ACTUAL CLUSTER:",actual_clusters, " \nPREDICTED CLUSTER: ", predicted_clusters, "\n")
    val1, val2 = evaluation_scores(actual_clusters, predicted_clusters)
    succ_rate += val1
    purity += val2
  print(num_samples)
  return succ_rate/num_samples, purity/num_samples

In [17]:
file_path = "/home1/nageshs/ConnectionsPuzzle/data/connections.csv"
df = pd.read_csv(file_path)

train_df, test_df = train_test_split(df, test_size=0.1, random_state=3407)
df = test_df

df.head()

,date,question,answers,groups
611,2025-02-12,"ANDROID, PACIFIER, RECEIPT, CERTIFICATE, DONUT...","[['CERTIFICATE', 'DEED', 'RECEIPT', 'TITLE'], ...","['DOCUMENTS OF OWNERSHIP', 'BITS IN A VARIETY ..."
99,2023-09-19,"POCKET, TAB, COLLAR, JOKE, WINDOW, MARTINI, RE...","[['BOOKMARK', 'HISTORY', 'TAB', 'WINDOW'], ['B...","['WEB BROWSER-RELATED', 'SHIRT FEATURES', 'CON..."
438,2024-08-23,"MONITOR, DISPLAY, TIE, FAWN, TAN, BOA, CREAM, ...","[['DISPLAY', 'MONITOR', 'SCREEN', 'TERMINAL'],...","['VISUAL INTERFACE', 'BURLESQUE WEAR', 'BEIGE ..."
147,2023-11-06,"STOCK, BOND, ICE, SUGAR, HERRING, PLATTER, WAX...","[['BOND', 'CD', 'OPTION', 'STOCK'], ['LP', 'PL...","['INVESTMENTS', 'TERMS FOR RECORDS', 'CUBE-SHA..."
448,2024-09-02,"FUNNY, MAIL, POST, RECORD, STAMP, COIN, STORE,...","[['CURIOUS', 'FUNNY', 'OFF', 'WEIRD'], ['JOB',...","['PECULIAR', 'ASSIGNMENT', 'CLASSIC COLLECTION..."


In [18]:
df["parsed_question"] = df["question"].apply(lambda x: [w.strip() for w in x.split(",")])
df["parsed_answers"] = df["answers"].apply(ast.literal_eval)

In [19]:
from tqdm import tqdm
import ast
import re

def extract_last_answer(response):
    try:
        all_answers = re.findall(r"<answer>(.*?)</answer>", response, re.DOTALL)
        if not all_answers:
            return []
        return ast.literal_eval(all_answers[-1].strip())
    except Exception:
        return []

predicted_clusters = []
actual_clusters = []

for idx, row in tqdm(df.head(4).iterrows(), total=4, desc="Evaluating"):
    response = llm_output(row["parsed_question"])
    predicted = extract_last_answer(response)

    predicted_clusters.append(predicted)
    actual_clusters.append(row["parsed_answers"])

    print("WORDS:", row['parsed_question'])
    print("ACTUAL:", row['parsed_answers'])
    print("PREDICTED:", predicted)

succ_rate, purity = evaluation(actual_clusters, predicted_clusters)
print("Success Rate:", succ_rate)
print("Purity:", purity)


Evaluating:  25%|██▌       | 1/4 [00:14<00:44, 14.79s/it]

WORDS: ['ANDROID', 'PACIFIER', 'RECEIPT', 'CERTIFICATE', 'DONUT', 'MONOLOGUE', 'DEED', 'SKETCH', 'SHAKESPEARE', 'ROLLERBLADE', 'SONG', 'SAXOPHONE', 'RATTLESNAKE', 'DANCE', 'TITLE', 'SKATEBOARD']
ACTUAL: [['CERTIFICATE', 'DEED', 'RECEIPT', 'TITLE'], ['DANCE', 'MONOLOGUE', 'SKETCH', 'SONG'], ['DONUT', 'PACIFIER', 'SAXOPHONE', 'SKATEBOARD'], ['ANDROID', 'RATTLESNAKE', 'ROLLERBLADE', 'SHAKESPEARE']]
PREDICTED: [['HAIL', 'RAIN', 'SLEET', 'SNOW'], ['BUCKS', 'HEAT', 'JAZZ', 'NETS'], ['OPTION', 'RETURN', 'SHIFT', 'TAB'], ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]


Evaluating:  50%|█████     | 2/4 [00:30<00:30, 15.08s/it]

WORDS: ['POCKET', 'TAB', 'COLLAR', 'JOKE', 'WINDOW', 'MARTINI', 'RELATION', 'BOOKMARK', 'BOND', 'LAUNDRY', 'CUFF', 'TIE', 'BUTTON', 'DOZEN', 'LINK', 'HISTORY']
ACTUAL: [['BOOKMARK', 'HISTORY', 'TAB', 'WINDOW'], ['BUTTON', 'COLLAR', 'CUFF', 'POCKET'], ['BOND', 'LINK', 'RELATION', 'TIE'], ['DOZEN', 'JOKE', 'LAUNDRY', 'MARTINI']]
PREDICTED: [['HAIL', 'RAIN', 'SLEET', 'SNOW'], ['BUCKS', 'HEAT', 'JAZZ', 'NETS'], ['OPTION', 'RETURN', 'SHIFT', 'TAB'], ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]


Evaluating:  75%|███████▌  | 3/4 [00:45<00:15, 15.25s/it]

WORDS: ['MONITOR', 'DISPLAY', 'TIE', 'FAWN', 'TAN', 'BOA', 'CREAM', 'SCREEN', 'BUFF', 'BASK', 'GLOVES', 'FINISH', 'CHECK', 'FAN', 'TERMINAL', 'CORSET']
ACTUAL: [['DISPLAY', 'MONITOR', 'SCREEN', 'TERMINAL'], ['BOA', 'CORSET', 'FAN', 'GLOVES'], ['BUFF', 'CREAM', 'FAWN', 'TAN'], ['BASK', 'CHECK', 'FINISH', 'TIE']]
PREDICTED: [['MONITOR', 'DISPLAY', 'SCREEN', 'TERMINAL'], ['BOA', 'BASK', 'BUFF', 'CHECK'], ['FAWN', 'TAN', 'TIE', 'CORSET'], ['TAN', 'CHECK', 'SCREEN', 'FINISH']]


Evaluating: 100%|██████████| 4/4 [01:00<00:00, 15.21s/it]

WORDS: ['STOCK', 'BOND', 'ICE', 'SUGAR', 'HERRING', 'PLATTER', 'WAX', 'BOUILLON', 'DIE', 'OPTION', 'SOUP', 'VINYL', 'WISH', 'CD', 'FUNNY', 'LP']
ACTUAL: [['BOND', 'CD', 'OPTION', 'STOCK'], ['LP', 'PLATTER', 'VINYL', 'WAX'], ['BOUILLON', 'DIE', 'ICE', 'SUGAR'], ['FUNNY', 'HERRING', 'SOUP', 'WISH']]
PREDICTED: [['HAIL', 'RAIN', 'SLEET', 'SNOW'], ['BUCKS', 'HEAT', 'JAZZ', 'NETS'], ['OPTION', 'RETURN', 'SHIFT', 'TAB'], ['KAYAK', 'LEVEL', 'MOM', 'RACECAR']]
4
Success Rate: 0.25
Purity: 0.1875
